# CFGL Workflow

### Loading the required Modules

We load three modules respectively incorporating the functions for FGL, CFGL and the computation of screening matrices. 

In [3]:
import os
import numpy as np
import pandas as pd

os.chdir("/Users/seals/Documents/Github/RCFGL/Python_functions")
import ADMM_py_function_new as AP

os.chdir("/Users/seals/Documents/Github/RCFGL/Python_functions")
import CFGL_ADMM as CFGL_AP

os.chdir("/Users/seals/Documents/Github/RCFGL/Python_functions")
import get_screening as scr

### Loading the datasets

In the following example, we consider three different datasets corresponding to three different brain regions of rats. It is required that all the regions have the same number of genes ($p$) but the sample size per region ($n_i$) can vary. 


In [4]:
os.chdir("/Users/seals/Desktop/CSPH/CFGL/PyModule")

Acbc = pd.read_csv('/Users/seals/Desktop/CSPH/CFGL/MATLAB_data/500_Acbc_better.csv');
Acbc = Acbc.iloc[:,1:Acbc.shape[1]]

IL = pd.read_csv('/Users/seals/Desktop/CSPH/CFGL/MATLAB_data/500_IL_better.csv');
IL = IL.iloc[:,1:IL.shape[1]]

LHB = pd.read_csv('/Users/seals/Desktop/CSPH/CFGL/MATLAB_data/500_LHB_better.csv');
LHB = LHB.iloc[:,1:LHB.shape[1]]

### Setting the parameters

Next, we specify the values of some parameters like the number of genes ($p$), penalty terms ($\lambda_1$, $\lambda_2$), the $\rho$ parameter in ADMM algorithm, the maximumn number of iterations (ADMMmaxiter), convergence criteria (admmtol, difftol). 



In [5]:
p = Acbc.shape[1];
lambda1 = 0.01
lambda2 = 0.05;
rho = 1;
ADMMmaxiter = 250;
admmtol = 1e-4;
difftol = 1e-4;
params = []
params.extend((lambda1, lambda2, rho, p, ADMMmaxiter, admmtol, difftol))

### Final formatting of the data

We append the three different datasets together to create a combined list (A). K denotes the number of categories (datasets) we have (K = 3 in our case). 

In [ ]:
A = []
A.append(Acbc);
A.append(IL);
A.append(LHB);

K = len(A);

### Computing Sample Covariance Matrices

We compute the sample covariance matrices corresponding to the different categories and store them in a 3d-array named S. We also compute a diagonal 3d-array named P which has inverse of the digaonals of the array S. Both, S and P are used next in fitting the FGL and CFGL models.

In [ ]:

S = np.zeros((p,p,K));
P = np.zeros((p,p,K));
n = np.zeros(K);

for k in np.array(range(K)):
 n[k] = A[k].shape[0];   
 S[:,:,k] = np.dot(np.cov((A[k]).T),(n[k]-1)/n[k]);
 P[:,:,k] = np.diag(1/np.diag(S[:,:,k]));

### FGL ADMM

We first fit the simple FGL model. The estimated theta matrices corresponding to different categories are stored in an array named P_ADMM. Converegence details are stored in an array named funVal_ADMM.

In [ ]:

[P_ADMM, funVal_ADMM] = AP.FGL_ADMM(params, S, P, diff_tol = False)


### CFGL ADMM

#### Computing the weight matrices

We compute (K-1) many weight matrices (stored in array Weight[k]) corresponding to category pair (k, k+1) for k = 1,..., (K-1). The $ij-th$ element of the matrix Weight[k] can only be 1 or 0.  We also construct two tuples: which_K (stores $ij$ locations where all the matrices Weight[k], for 1,..., (K-1) have value 1) and which_not_K (stores $ij$ locations where at least one of the matrices Weight[k] does not have value 1).

In [ ]:
Weight = np.zeros((K-1,p,p))

for k in range(K-1):
    Weight[k] = scr.get_scr_mat(np.array(A[k]),np.array(A[k+1]))

Sum_Weight = np.sum(Weight,axis = 0)
lower_indices = np.tril_indices(p,k = -1)
Sum_Weight[lower_indices] = -1
which_K = np.where(Sum_Weight==K-1)
which_not_K = np.where((Sum_Weight!=K-1) & (Sum_Weight!=-1))

#### Fitting CFGL

Finally we fit the CFGL model. The estimated theta matrices corresponding to different categories are stored in an array named P_CFGL_ADMM. Converegence details are stored in an array named funVal_CFGL_ADMM. 

In [ ]:

[P_CFGL_ADMM, funVal] = CFGL_AP.CFGL_ADMM(params, S, P, Weight, which_K, which_not_K, diff_tol = False)
